In [ ]:
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

import pytz

import astroplan
from astroplan import Observer, FixedTarget, ObservingBlock, SequentialScheduler, Transitioner

from astroplan import plots
color_cycle = plots.mplstyles.astropy_mpl_style['axes.color_cycle']

In [ ]:
pal = Observer.at_site('Palomar', timezone=pytz.timezone('US/Pacific'))
pal

In [ ]:
midnight = pal.midnight(Time('2015-11-03 5:00'))
midnight

In [ ]:
targets = [FixedTarget.from_name('Vega'),
           FixedTarget.from_name('Deneb'),
           FixedTarget.from_name('Altair'),
           FixedTarget.from_name('Albireo')]
targets

In [ ]:
t1 = pal.twilight_evening_astronomical(midnight)
ts = t1 + [0, 1, 2, 3]*u.hour
for target, c in zip(targets, color_cycle):
    plots.plot_sky(target, pal, ts, style_kwargs=dict(color=c))
plt.plot(np.linspace(0,2*np.pi,100), [np.degrees(np.arccos(1/2))]*100, color='r')
plt.plot(np.linspace(0,2*np.pi,100), [np.degrees(np.arccos(1/3))]*100, color='r')
plt.legend(loc='upper left')

In [ ]:
etime = 5*u.min
n = 10
rot = 55*u.second

blocks = [ObservingBlock.from_exposures(t, etime, n, rot) for t in targets]

In [ ]:
blocks

In [ ]:
constraints = [astroplan.constraints.AirmassConstraint(3, boolean_constraint=False),
               astroplan.constraints.AtNightConstraint(18*u.deg)
               #,astroplan.constraints.MoonSeparationConstraint(5*u.deg)
               ]
trans = Transitioner(slew_rate=1*u.deg/u.second)

In [ ]:
start_time = pal.twilight_evening_astronomical(midnight)
end_time = pal.twilight_morning_astronomical(midnight)
scheduler = SequentialScheduler(start_time, end_time,
                                constraints=constraints, observer=pal,
                                transitioner=trans)
schedule = scheduler(blocks)
for block in schedule:
    local_start = pal.astropy_time_to_datetime(block.start_time)
    if hasattr(block, 'target'):
        print(block.target.name, 'starting @', local_start, 'lasting', block.duration)
    else:
        print('trans starting @', local_start, 'lasting', block.duration, ':', block.components)

In [ ]:
[b.constraints_value if hasattr(b, 'constraints_value') else None for b in schedule]

In [ ]:
ts = start_time + np.linspace(0, (end_time-start_time).value, 100)*u.day
targ_to_color = {}
for target, c in zip(targets, color_cycle):
    plots.plot_airmass(target, pal, ts, style_kwargs=dict(color=c))
    targ_to_color[target.name] = c
plt.legend(loc=0)

for block in schedule:
    if hasattr(block, 'target'):
        plt.axvspan(block.start_time.plot_date, block.end_time.plot_date, 
                    fc=targ_to_color[block.target.name], lw=0, alpha=.2)
ax=plt.gca()